In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CLASS")

LangSmith 추적을 시작합니다.
[프로젝트명]
CLASS


In [3]:
from langchain_community.document_loaders import PDFPlumberLoader

In [4]:
# pip install pdfplumber

## 전처리 함수

In [ ]:
# import re
# from PyPDF2 import PdfReader

# def remove_wn_from_text(text):
#     """
#     텍스트에서 'Wn'을 제거하는 함수
#     """
#     return re.sub(r'Wn', '', text)

# def clean_text(text):
#     """
#     텍스트 정리 함수
#     - \n 제거
#     - >>> 제거
#     - 불필요한 공백 제거
#     """
#     text = text.replace('\n', ' ')  # \n 제거
#     text = re.sub(r'>>>', '', text)  # >>> 제거
#     text = re.sub(r'\s+', ' ', text)  # 여러 공백을 하나의 공백으로 변경
#     return text.strip()

# # PDF 파일 읽기
# reader = PdfReader("./data/converted_data_test_with_metadata.pdf")
# text = ""
# for page in reader.pages:
#     text += page.extract_text()

# # Wn 제거 및 텍스트 정리
# text = remove_wn_from_text(text)
# cleaned_text = clean_text(text)

# # 결과 저장
# with open("./data/cleaned_data_test_with_metadata.txt", "w", encoding="utf-8") as file:
#     file.write(cleaned_text)


In [5]:
loader = PDFPlumberLoader('./data/converted_data_test_with_metadata.pdf')

In [6]:
docs = loader.load()

In [7]:
len(docs)

16

In [8]:
docs

[Document(metadata={'source': './data/converted_data_test_with_metadata.pdf', 'file_path': './data/converted_data_test_with_metadata.pdf', 'page': 0, 'total_pages': 16, 'Producer': 'PyPDF2', 'Title': 'Jump to Phython', 'Author': 'Park', 'Subject': 'Python', 'Keywords': 'Python, Book', 'Creator': 'PyPDF2 Library'}, page_content="1장\n01장 파이썬이란 무엇인가?\n필자는 파이썬의 프롬프트(>>>)를 처음 본 순간부터 지금까지 줄곧 파이썬과 함께 지내\n온 듯하다. ‘프로그래밍은 어렵고 지루하다’라는 고정관념을 가지고 있던 필자에게 파이\n썬은 커다란 충격으로 다가왔다. 여러분도 이 책을 통해 파이썬의 매력에 흠뻑 빠져 보\n기를 바란다. 01장에서는 파이썬의 특징과 장단점을 알아보고 파이썬 프로그래밍을 위한\n환경 구축 방법에 대해 배운다. 그리고 간단한 파이썬 프로그램도 작성해 본다.\n01-1 파이썬이란?\n파이썬(Python)은 1990년 암스테르담의 귀도 반 로섬(Guido van rossum)이 개발한 인터\n프리터 언어이다. 귀도는 파이썬이라는 이름을 자신이 좋아하는 코미디 쇼인 ‘몬티 파이\n썬의 날아다니는 서커스(Monty python's flying circus)’에서 따왔다고 한다.\n인터프리터 언어란 소스 코드를 한 줄씩 해석한 후 그때그때 실행해 결과를 바로 확인할\n수 있는 언어를 말한다.\n파이썬의 사전적 의미는 ‘고대 신화에 나오는 파르나소스 산의 동굴에 살던 큰 뱀’을 뜻\n하며, 아폴로 신이 델파이에서 파이썬을 퇴치했다는 이야기가 전해지고 있다. 대부분의\n파이썬 책 표지와 아이콘이 뱀 모양으로 그려져 있는 이유는 바로 이 때문이다.\n파이썬은 컴퓨터 프로그래밍을 교육할 때

In [9]:
for doc in docs:
    doc.metadata['filename'] = doc.metadata['source']

In [10]:
# !pip install -q ragas==0.1.19

In [11]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\RMARKET\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# 생성기
generator_llm = ChatOpenAI(model = 'gpt-4o')

In [13]:
# 평가를 할 수 있는 비평기
critic_llm = ChatOpenAI(model = 'gpt-4o')

In [14]:
# 임베딩 모델
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [15]:
# 텍스트 분할기
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

In [16]:
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model = 'gpt-4o'))
# 구문추출기 생성을 위한 모델 호출(RAGAS와의 호환을 위한 Wrapper)

In [17]:
# 구문추출기 : 문서 핵심 정보 식별 및 추출 역할
Keyphrase_extractor = KeyphraseExtractor(llm = langchain_llm)

In [18]:
# 임베딩 모델도 RAGAS와 호환을 위해 Wrapper 적용
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

In [19]:
# 저장소, docstore 만들기
docstore = InMemoryDocumentStore(
    splitter = splitter,
    embeddings = ragas_embeddings,
    extractor = Keyphrase_extractor
)

In [20]:
# 생성기 만들기
generator = TestsetGenerator.from_langchain(
    generator_llm, # 생성기
    critic_llm,  # 판별기
    ragas_embeddings, # 임베딩모델
    docstore = docstore # 문서저장소
)

In [21]:
# # 질문 유형 분포를 설정하는 딕셔너리입니다.
# # 각 질문 유형과 그에 해당하는 비율(0~1)을 정의합니다.
# distributions = {
#     "simple": 0.4,          # 단순한 질문 유형의 비율 (전체의 40%)
#     "reasoning": 0.2,       # 논리적 사고가 필요한 질문 유형의 비율 (전체의 20%)
#     "multi_context": 0.2,   # 여러 문맥을 고려해야 하는 질문 유형의 비율 (전체의 20%)
#     "conditional": 0.2      # 조건에 따라 답변이 달라지는 질문 유형의 비율 (전체의 20%)
# }

In [22]:
# 질문 유형 분포 결정
distributions = {simple : 0.5, reasoning : 0.2, multi_context : 0.2, conditional : 0.1}

## 배치 사이즈

In [23]:
# def batch_documents(documents, batch_size):
#     """
#     문서를 일정 크기의 배치로 나눕니다.
    
#     Args:
#         documents (list): 문서 목록.
#         batch_size (int): 각 배치의 크기.
    
#     Returns:
#         list: 배치로 나뉜 문서 그룹.
#     """
#     for i in range(0, len(documents), batch_size):
#         yield documents[i:i + batch_size]

# def generate_testset_in_batches(generator, documents, test_size, distributions, batch_size=10):
#     """
#     문서를 배치 단위로 나누어 처리하여 테스트 세트를 생성합니다.
    
#     Args:
#         generator: 테스트 세트 생성기 객체.
#         documents (list): 전체 문서 목록.
#         test_size (int): 각 배치에서 생성할 질문 세트 크기.
#         distributions (dict): 질문 유형 분포.
#         batch_size (int): 배치 크기 (기본값: 10).
    
#     Returns:
#         list: 각 배치의 결과를 결합한 전체 테스트 세트.
#     """
#     all_testsets = []
#     for batch_index, document_batch in enumerate(batch_documents(documents, batch_size)):
#         print(f"Processing batch {batch_index + 1}/{len(documents) // batch_size + 1}...")
#         try:
#             batch_testset = generator.generate_with_langchain_docs(
#                 documents=document_batch,
#                 test_size=test_size,
#                 distributions=distributions,
#                 with_debugging_logs=True,
#                 raise_exceptions=False
#             )
#             all_testsets.extend(batch_testset.to_dict())  # 배치 결과를 결합
#         except Exception as e:
#             print(f"Error processing batch {batch_index + 1}: {e}")
    
#     return all_testsets

# # 예제 사용
# testset = generate_testset_in_batches(
#     generator=generator,
#     documents=docs,  # 전체 문서
#     test_size=10,    # 각 배치에서 생성할 테스트 세트 크기
#     distributions=distributions,
#     batch_size=50    # 배치 크기
# )

# # 결과를 pandas DataFrame으로 변환
# import pandas as pd
# test_df = pd.DataFrame(testset)

# # DataFrame 확인
# print(test_df.head())


In [ ]:
testset = generator.generate_with_langchain_docs(
    documents = docs,
    test_size = 10,
    distributions = distributions,
    with_debugging_logs = True,
    # 로그 활성화
    raise_exceptions= False
    # 예외가 발생해도 계속 진행, 예외 발생 시 문서가 처리 되지 않거나 문서에 기록됨
)

embedding nodes:  91%|█████████ | 40/44 [00:02<00:00, 13.65it/s]Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Monthly unique traces usage limit exceeded"}')trace=9ccb5452-d7b7-4e54-91eb-f1a6d46806af,id=9ccb5452-d7b7-4e54-91eb-f1a6d46806af; trace=655e5895-09ac-48f3-9b2e-3c0f0d7db2ae,id=655e5895-09ac-48f3-9b2e-3c0f0d7db2ae; trace=d83b90dd-0162-4895-b2e3-8c7dc6b3d7aa,id=d83b90dd-0162-4895-b2e3-8c7dc6b3d7aa; trace=30fe328e-ec3e-40ee-8c94-78d84f9e6846,id=30fe328e-ec3e-40ee-8c94-78d84f9e6846
Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Monthly unique traces usage l

In [25]:
test_df = testset.to_pandas()
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What steps are involved in executing a program...,"[IDLE 셸은 IDLE 에디터에서 실행한 프로그램의 결과가 표시되는 창으로, 앞에...",To execute a program using the 'Run Module' op...,simple,[{'source': './data/converted_data_test_with_m...,True
1,\ud30c\uc774\uc36c\uc744 \uc0ac\uc6a9\ud558\ub...,[1장\n01장 파이썬이란 무엇인가?\n필자는 파이썬의 프롬프트(>>>)를 처음 본...,"Python is widely used in many IT companies, in...",simple,[{'source': './data/converted_data_test_with_m...,True
2,How can C and C++ programming be utilized in p...,[있기 때문이다. C나 C++로 만든 프로그램을 파이썬에서 사용할 수 있고 파이썬으...,C and C++ programming can be utilized in proje...,simple,[{'source': './data/converted_data_test_with_m...,True
3,How is Python utilized in the field of the Int...,[있기 때문이다. C나 C++로 만든 프로그램을 파이썬에서 사용할 수 있고 파이썬으...,Python is utilized in the field of the Interne...,simple,[{'source': './data/converted_data_test_with_m...,True
4,What steps are involved in executing a program...,"[IDLE 셸은 IDLE 에디터에서 실행한 프로그램의 결과가 표시되는 창으로, 앞에...",To execute a program using the 'Run Module' op...,simple,[{'source': './data/converted_data_test_with_m...,True
5,Why add Py ext. before using Py in V. Studio?,[운로드 사이트에서 내려받을 수 있다.\n비주얼 스튜디오 코드는 파이썬 전용 에디터...,The first thing you need to do to use Python i...,reasoning,[{'source': './data/converted_data_test_with_m...,True
6,What in Python helps quick learning and aligns...,[• 파이썬은 인간다운 언어이다\n• 문법이 쉬워 빠르게 배울 수 있다\n• 무료이...,Python is a human-friendly language with simpl...,reasoning,[{'source': './data/converted_data_test_with_m...,True
7,How has Python's appeal shaped its role in tec...,[1장\n01장 파이썬이란 무엇인가?\n필자는 파이썬의 프롬프트(>>>)를 처음 본...,Python is widely used in tech firms due to its...,multi_context,[{'source': './data/converted_data_test_with_m...,True
8,How do Python annotations help in code clarity...,"[IDLE 셸은 IDLE 에디터에서 실행한 프로그램의 결과가 표시되는 창으로, 앞에...",The answer to given question is not present in...,multi_context,[{'source': './data/converted_data_test_with_m...,True
9,How to run Python if IDLE isn't available?,[상 .py로 해야 한다. .py는 파이썬 파일이라는 것을 알려 주는 확장자이다.\...,To run a Python program if IDLE isn't availabl...,conditional,[{'source': './data/converted_data_test_with_m...,True


In [26]:
df = test_df

In [27]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What steps are involved in executing a program...,"[IDLE 셸은 IDLE 에디터에서 실행한 프로그램의 결과가 표시되는 창으로, 앞에...",To execute a program using the 'Run Module' op...,simple,[{'source': './data/converted_data_test_with_m...,True
1,\ud30c\uc774\uc36c\uc744 \uc0ac\uc6a9\ud558\ub...,[1장\n01장 파이썬이란 무엇인가?\n필자는 파이썬의 프롬프트(>>>)를 처음 본...,"Python is widely used in many IT companies, in...",simple,[{'source': './data/converted_data_test_with_m...,True
2,How can C and C++ programming be utilized in p...,[있기 때문이다. C나 C++로 만든 프로그램을 파이썬에서 사용할 수 있고 파이썬으...,C and C++ programming can be utilized in proje...,simple,[{'source': './data/converted_data_test_with_m...,True
3,How is Python utilized in the field of the Int...,[있기 때문이다. C나 C++로 만든 프로그램을 파이썬에서 사용할 수 있고 파이썬으...,Python is utilized in the field of the Interne...,simple,[{'source': './data/converted_data_test_with_m...,True
4,What steps are involved in executing a program...,"[IDLE 셸은 IDLE 에디터에서 실행한 프로그램의 결과가 표시되는 창으로, 앞에...",To execute a program using the 'Run Module' op...,simple,[{'source': './data/converted_data_test_with_m...,True


In [46]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})

In [47]:
import ast
def convert_to_list(example):
    contexts = ast.literal_eval(example['contexts'])
    return {'contexts':contexts}

In [48]:
# test_dataset = test_dataset.map(convert_to_list)

In [49]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [50]:
loader = PyMuPDFLoader('data/converted_data_with_metadata.pdf')
docs = loader.load()

In [51]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
split_documents = text_splitter.split_documents(docs)

In [52]:
embeddings = OpenAIEmbeddings()

In [53]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [54]:
retriever = vectorstore.as_retriever()

In [71]:
prompt = PromptTemplate.from_template(
    """당신은 질문에 답변을 수행하는 파이썬 관련 용어 사전을 제공하는 학습 비서입니다. 
    당신은 주어진 context에서 주어진 question에 답하는 것을 수행합니다. 검색된 결과인 다음 context를 사용하여 질문인 question에 답하세요.
    만약, context에서 답을 찾을 수 없거나, 답을 모른다면 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.'라고 한글로 질문하고 답변해 주세요.
    이름이나 기술적인 용어는 번역하지 않고 그대로 출력해주세요.
    
    
    # Context : {context}
    # Question : {question}
    # Answer :
    """
)

In [72]:
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

In [73]:
chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [74]:
batch_dataset = []
for question in test_dataset['question']:
    batch_dataset.append(question)

In [75]:
batch_dataset[:3]

['What steps are involved in executing a program using the "Run Module" option in IDLE?',
 '\\ud30c\\uc774\\uc36c\\uc744 \\uc0ac\\uc6a9\\ud558\\uba74 \\ud504\\ub85c\\uadf8\\ub798\\ubc0d \\uac1c\\ubc1c\\uc5d0 \\uc5b4\\ub5a4 \\ucd94\\uac00 \\uac00\\uce58\\uac00 \\uc788\\ub294\\uac00?',
 'How can C and C++ programming be utilized in projects involving Raspberry Pi and the Internet of Things?']

In [76]:
answer = chain.batch(batch_dataset)
answer[:3]

['IDLE에서 "Run Module" 옵션을 사용하여 프로그램을 실행하는 단계는 다음과 같습니다:\n\n1. IDLE 에디터를 실행합니다.\n2. 메뉴에서 [File → New File]을 선택하여 빈 창을 엽니다.\n3. 파이썬 프로그램을 작성합니다. 예를 들어, `# hello.py`와 같은 주석과 함께 `print("Hello World")`를 작성합니다.\n4. 프로그램을 저장합니다. 파일 이름의 확장자는 항상 .py로 해야 하며, 적절한 디렉터리에 저장합니다.\n5. 메뉴에서 [Run → Run Module]을 선택하거나 단축키 F5를 눌러 프로그램을 실행합니다.\n6. 프로그램 실행 시 파일을 먼저 저장하라는 대화 상자가 나타나면 [확인]을 선택합니다.\n\n이 과정을 통해 IDLE에서 작성한 파이썬 프로그램을 실행할 수 있습니다.',
 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.',
 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.']

In [77]:
if 'answer' in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(['answer']).add_column('answer', answer)
else:
    test_dataset = test_dataset.add_column('answer', answer)

In [78]:
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer'],
    num_rows: 10
})

In [79]:
from ragas import evaluate
from ragas.metrics import answer_relevancy,faithfulness,context_recall,context_precision

In [80]:
result = evaluate(
    dataset = test_dataset,
    metrics = [
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision
    ]
)


# answer_relevancy : 모델이 문맥에서 중요한 정보를 잘 회상, 재현했는가
# faithfulness : 모델의 답변이 문맥에 기반해서 사실적인가(factual)
# context_recall : 전체 질문에 대해 모델의 답변이 질문과 얼마나 관련성이 있는가
# context_precision : 모델이 문맥에서 필요한 정보를 정확히 활용했는가

Evaluating: 100%|██████████| 40/40 [00:51<00:00,  1.29s/it]


In [81]:
result

{'answer_relevancy': 0.2815, 'faithfulness': 0.3714, 'context_recall': 0.3500, 'context_precision': 0.8000}

In [84]:
result_df = result.to_pandas()
result_df

,question,contexts,answer,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What steps are involved in executing a program...,"[IDLE 셸은 IDLE 에디터에서 실행한 프로그램의 결과가 표시되는 창으로, 앞에...","IDLE에서 ""Run Module"" 옵션을 사용하여 프로그램을 실행하는 단계는 다음...",To execute a program using the 'Run Module' op...,0.962739,1.000000,1.0,1.0
1,\ud30c\uc774\uc36c\uc744 \uc0ac\uc6a9\ud558\ub...,[1장\n01장 파이썬이란 무엇인가?\n필자는 파이썬의 프롬프트(>>>)를 처음 본...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,"Python is widely used in many IT companies, in...",0.000000,0.000000,0.0,1.0
2,How can C and C++ programming be utilized in p...,[있기 때문이다. C나 C++로 만든 프로그램을 파이썬에서 사용할 수 있고 파이썬으...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,C and C++ programming can be utilized in proje...,0.000000,0.000000,0.5,0.0
3,How is Python utilized in the field of the Int...,[있기 때문이다. C나 C++로 만든 프로그램을 파이썬에서 사용할 수 있고 파이썬으...,파이썬은 사물 인터넷(Internet of Things) 분야에서도 활용도가 높습니...,Python is utilized in the field of the Interne...,0.894766,1.000000,0.0,1.0
4,What steps are involved in executing a program...,"[IDLE 셸은 IDLE 에디터에서 실행한 프로그램의 결과가 표시되는 창으로, 앞에...","IDLE에서 ""Run Module"" 옵션을 사용하여 프로그램을 실행하는 단계는 다음...",To execute a program using the 'Run Module' op...,0.957441,1.000000,1.0,1.0
5,Why add Py ext. before using Py in V. Studio?,[운로드 사이트에서 내려받을 수 있다.\n비주얼 스튜디오 코드는 파이썬 전용 에디터...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The first thing you need to do to use Python i...,0.000000,0.000000,0.0,1.0
6,What in Python helps quick learning and aligns...,[• 파이썬은 인간다운 언어이다\n• 문법이 쉬워 빠르게 배울 수 있다\n• 무료이...,파이썬의 문법이 매우 쉽고 간결하여 사람의 사고 체계와 매우 닮아 있기 때문에 빠르...,Python is a human-friendly language with simpl...,0.000000,0.714286,0.0,1.0
7,How has Python's appeal shaped its role in tec...,[1장\n01장 파이썬이란 무엇인가?\n필자는 파이썬의 프롬프트(>>>)를 처음 본...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,Python is widely used in tech firms due to its...,0.000000,0.000000,0.0,1.0
8,How do Python annotations help in code clarity...,"[IDLE 셸은 IDLE 에디터에서 실행한 프로그램의 결과가 표시되는 창으로, 앞에...",주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The answer to given question is not present in...,0.000000,0.000000,1.0,0.0
9,How to run Python if IDLE isn't available?,[상 .py로 해야 한다. .py는 파이썬 파일이라는 것을 알려 주는 확장자이다.\...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,To run a Python program if IDLE isn't availabl...,0.000000,0.000000,0.0,1.0


In [85]:

result_df.loc[:,'context_recall' : 'context_precision']

,context_recall,context_precision
0,1.0,1.0
1,0.0,1.0
2,0.5,0.0
3,0.0,1.0
4,1.0,1.0
5,0.0,1.0
6,0.0,1.0
7,0.0,1.0
8,1.0,0.0
9,0.0,1.0


|context_precision (문맥 정밀도)|faithfulness (신뢰성)|answer_relevancy (답변 적합성)|context_recall (문맥 재현율)|
|---|---|---|---|
|답변에 사용된 문맥이 정확했는지|답변이 문맥에 기반해 사실적(factual)인지|답변이 질문과 관련성이 있는지|답변이 문맥에서 얼마나 많은 정보를 활용했는지|
|1.0: 답변에 사용된 정보가 문맥에서 매우 정확하게 활용됨|1.0: 답변이 문맥과 매우 정확하게 일치|1.0: 답변이 질문과 매우 높은 관련성을 가짐|1.0: 답변이 문맥의 모든 중요한 정보를 잘 회상|
|0.0: 답변이 문맥과 무관하거나, 부정확한 정보를 활용|0.0: 답변이 문맥에 기반하지 않고, 사실과 다를 가능성이 큼|0.0: 답변이 질문과 관련이 없거나, 부정확|0.0: 답변이 문맥을 활용하지 못함|